- Truncate audio based on the TXT captions file
- Generate TXT and VTT files for Front-end

Execute after upload, not yet implemented

In [1]:
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO 
import os
import numpy as np
from scipy.io import wavfile
import pandas as pd
import datetime

/anaconda3/envs/analysis/lib/python3.7/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def covertToSecs(time_str):
    hr, mi, se = time_str.split(':')
    return int(hr)*3600 + int(mi)*60 + int(se)

In [3]:
"""
Required structure for the uploads folder:
-- /raw_data
---- [course]_raw.txt
---- [course].wav
---- /unsorted

"""

'\nRequired structure for the uploads folder:\n-- /raw_data\n---- [course]_raw.txt\n---- [course].wav\n---- /unsorted\n\n'

In [4]:
filename = 'JP1_sample'

In [7]:
def process_uploads(filename):
    path = '../raw_data/'+filename+'/'
    caption_file = path+filename+'_subs.txt' # uploaded
    audio_file = path+filename+'.wav' # uploaded
    output_audio_path = path+'unsorted/'
    output_txt_path = path+'display/'
    
    fs, s = wavfile.read(audio_file) 
    duration = s.shape[0]/fs
    audio_segs = {}
    captions_tmp = []
    with open(caption_file,'r') as f:
        captions_tmp = f.readlines()

    captions = {} # extract captions
    captions_raw = {}
    for line in captions_tmp[1:]: # skip the header
        tm,sub = line.split('\t', 1) # it was \t not space
        start = covertToSecs(tm)
        captions[start] = sub.strip()
        captions_raw[tm] = sub.strip()
    
    audio_segs = {}
    tms = list(captions.keys())
    for i,tm in enumerate(tms):
        if i < len(tms)-1:
            audio_segs[tm] = s[tms[i] * fs: (tms[i+1]) * fs]
        else:
            audio_segs[tm] = s[tms[i] * fs: ]

    for tm,seg in audio_segs.items():
        wavfile.write(output_audio_path+str(tm)+'.000'+'.wav', fs, seg)
    
    # To generate VTT file
    with open(output_txt_path+filename+'.vtt','w') as f:
        f.write('WEBVTT\n\n')

        tms = list(captions_raw.keys())
        for i,tm in enumerate(tms):
            if i < len(tms)-1:
                f.write("%s.000 --> %s.000\n"%(tms[i],tms[i+1]))
                f.write(captions_raw[tms[i]]+'\n\n')
            else:
                f.write("%s.000 --> %s\n"%(tms[i], str(datetime.timedelta(seconds=duration))))
                f.write(captions_raw[tms[i]]+'\n\n')
                
    # To generate TXT file
    with open(output_txt_path+filename+'.txt','w') as f:
        tms = list(captions.keys())
        f.write('0.000 %s.000\n'%tms[0])
        for i,tm in enumerate(tms):
            if i < len(tms)-1:
                f.write("%s.000 %s.000\n"%(tms[i],tms[i+1]))
            else:
                f.write("%s.000 %.3f\n"%(tms[i], float(duration)))

In [8]:
process_uploads(filename)